# Текстовый калькулятор

Преснухин Д., Арясин Н., Марков В., ПИ21-2

In [1]:
def get_key(value):  # возвращает ключ по значению
    if value == '0000' or value == '000' or value == '00':  # чтобы не было лишних нулей в выводе
        return ''
    value = int(value)
    if floatt:  # для вещ. чисел сперва используем дополнительный словарь dic_fl
        d = {**dic_fl, **dic}
    else:  # иначе только основной словарь
        d = dic
    for k, v in d.items():  # цикл для поиска ключа по значению
        if v == value:
            return f'{k} '


def output3(x):
    global s
    l = len(x)
    output2(x[:l - 3])  # вызывем output2 для левой части переданного числа

    # дописывает "тысяч" с правильным окончанием
    if 11 <= int(x[l - 5:l - 3]) <= 19:
        ending = ''
    else:
        ending = thousand_endings[x[l - 4]]
    if s[-3:-1] != ' и':  # если есть хоть сколько-нибудь тысяч, дописываем
        s += f'тысяч{ending} '

    output2(x[l - 3:])  # вызывем output2 для правой части переданного числа (три последние цифры)


def output2(x):
    global s
    l = len(x)  # длина полученной строки цифр
    if l == 2:  # обработка чисел [11; 19]
        try:
            s += get_key(str(x))  # ищем при помощи get_key
            return
        except:
            pass
    if l <= 4:
        s += get_key(x[0] + '0' * (l - 1))  # берем первую цифру полученной строки и приписываем нули (для поиска по словарю)
        x = str(int(x) % int("1" + "0" * (l - 1)))  # убираем первую цифру
        if x == '0':
            return
        else:
            return output2(x)
    else:
        output3(x)  # для строки длиннее четырех (вывод тысяч)


def output(x):  # основная функция вывода
    global floatt, s  # объявляем нужные глобальные переменные
    s = ''  # то, куда будет записываться вывод
    point_index = x.find('.')  # пытаемся найти точку
    if x.find('.') != -1:  # если нашли точку
        floatt = True  # число вещественное - ДА
        output2(x[:point_index])  # переводим в текст целую часть

        # дописываем "целых и " с правильным окончанием
        try:
            if 11 <= int(x[point_index - 2:point_index]) <= 19:
                float_ending = 'ых'
            else:
                float_ending = float_endings[x[point_index - 1]]
        except:
            float_ending = float_endings[x[point_index - 1]]
        s += f"цел{float_ending} и "

        output2(x[point_index + 1:])  # переводим в текст дробную часть
    else:  # если число целое
        floatt = False
        output2(x)
    if floatt:  # если результат вещественный, нужно написать, что это за доли числа
        if len(x[point_index + 1:]) >= 2 and 11 <= int(x[-2:]) <= 19:  # проверка последних двух цифр, чтобы определить окончание
            ending = 'ых'
        else:
            ending = float_endings[x[-1]]
        l = len(x[point_index + 1:])
        if l == 6:
            s += 'миллионн'
        elif l == 5:
            s += 'стотысячн'
        elif l == 4:
            s += 'десятитысячн'
        elif l == 3:
            s += 'тысячн'
        elif l == 2:
            s += 'сот'
        elif l == 1:
            s += 'десят'
        s += ending
    return s


def calc(s):  # функция, которая переведет текстовое выражение в выражение с числами и найдет результат
    s_copy = s  # копируем s (введенную строку) для вывода в конце
    s = s.lower().split()  # преобразуем s в список
    if len(s) == 0:  # проверка, не пустой ли список
        return print('Вы ничего не ввели')
    for i in s:  # проверяем, нет ли в s слов, которые отсутствуют в словарях и exception_list'е
        if (i not in exception_list) and (i not in {**dic, **float_dic}.keys()):
            return print('Некорректный ввод')
    fl_iter = iter([float_dic[i] for i in s if i in float_dic.keys()])  # итератор для дробных чисел (пояснено дальше)
# в него подаются только значения из словаря float_dic, чтобы потом домножать на них соответствующие дробные части чисел
    s = [dic[i] for i in s if i in dic.keys()]  # заполняем s значениями из основного словаря по ключу (прим.: ["двадцать", "пять"] -> [20, 5])
    # print(s)
    left_index = 0  # индекс, с которого будем суммировать составные числа (прим.: [100, 80, 2, "+", 2] -> [182, "+", 2])
    s2 = []  # новый список, в который будут записываться уже просуммированные значения
    for i in range(len(s)):  # цикл для суммирования значений
        if type(s[i]) == str:  # если нашелся знак операции/скобка
            if s[i] == '(':  # чтобы перед скобкой ничего не суммировать (иначе получится лишкний ноль)
                left_index = i + 1
                s2.append(s[i])  # записываем в новый список скобку
                continue
            if s[i - 1] == ')' and i != 0:  # двигаем left_index, чтобы не суммировать знак после закрывающей скобки
                left_index = i + 1
                s2.append(s[i])  # снова записываем в новый список скобку
                continue
            else:
                s2.append(sum(s[left_index:i]))  # вот теперь суммируем все числа между двумя знаками операций
                s2.append(s[i])  # записываем в новый список знак операции
                left_index = i + 1
        # print(s2)
    if s[-1] != ')':  # если на конце нет скобки, суммируем (в цикле такой вариант не учитывается, т.к. там сумма происходит, если находится тип 'str', а в конце всей строки нет ничео в str)
        s2.append(sum(s[left_index:]))
    # print(s2)
    for i in range(len(s2)):  # цикл, где будем учитывать дробные части (пока у нас дробные числа представлены в виде [102, '.', 1])
        if s2[i] == '.':  # если наткнулись на точку
            s2[i] = int(s2[i - 1]) + int(s2[i + 1]) * next(fl_iter)  # к тому, что слева от точки прибавляем то, что справа, умноженное на очередной элемент итератора
            # пример итератора: [..."милионных",... "десятых"...] -> iter([0.000001, 0.1])
            s2[i - 1] = 'delete'  # записываем в элементы слева и справа от точки "delete", чтобы потом не обавлять их в s3
            s2[i + 1] = 'delete'
    s3 = []  # список, в который скопируем все из s2, кроме элементов "delete"
    for i in range(len(s2)):  # вот в этом цикле и копируем
        if s2[i] != 'delete':
            s3.append(s2[i])
    s3 = ' '.join(map(str, s3))  # преобразуем список s3 в строку (прим.: "( 5 + 4 * 0.25 ) * 10"
    try:
        # при помощи встроенной функции eval пытаемся посчитать строку. Эта функция сразу учитывает порядок действий и поддерживает все заявленные нами операции. + округляем до миллионных
        result = round(eval(s3), 7)
    except ZeroDivisionError:
        return "Ошибка деления на ноль"
    except SyntaxError:
        return "Синтаксическая ошибка"
    except:
        return "Непредвиденная ошибка"
    result = f'{result:.6f}'  # форматируем число, чтобы не было чисел в экспоненциальной записи (1e-11, например)
    for i in range(len(result) - 1, -1, -1):  # убираем лишние незначащие нули справа, если они есть
        if result[i] == '.':
            result = result[:i]
            break
        if result[i] != '0':
            result = result[:i + 1]
            break
    # далее можно убрать решетку, чтобы дополнительно выводилось выражение и его результат в числовом виде
    print(f'{s3} = {result}')
    return print(f'{s_copy.capitalize()} = {output(result)}\n')  # выводим изначальное выражение = результат. Результат преобразуем в слова при помощи функции output


dic = {  # словарь с основными словами для ввода и вывода выражений
    "ноль": 0, "один": 1, "два": 2, "три": 3, "четыре": 4, "пять": 5, "шесть": 6, "семь": 7, "восемь": 8,
    "девять": 9, "десять": 10, "одиннадцать": 11, "двенадцать": 12, "тринадцать": 13, "четырнадцать": 14,
    "пятнадцать": 15, 'шестнадцать': 16, 'семнадцать': 17, 'восемнадцать': 18, "девятнадцать": 19,
    "двадцать": 20, "тридцать": 30, "сорок": 40, "пятьдесят": 50, "шестьдесят": 60, "семьдесят": 70,
    "восемьдесят": 80, "девяносто": 90, "сто": 100, "плюс": "+", "минус": "-", "умножить": "*", "разделить": "/",
    "остатком": "%", "открывается": "(", "закрывается": ")", "двести": 200, "триста": 300, "четыреста": 400,
    "пятьсот": 500, "шестьсот": 600, "семьсот": 700, "восемьсот": 800, "девятьсот": 900, "тысяча": 1000,
    "две тысячи": 2000, "три тысячи": 3000, "четыре тысячи": 4000, "пять тысяч": 5000, "шесть тысяч": 6000,
    "семь тысяч": 7000, "восемь тысяч": 8000, "девять тысяч": 9000, "одна": 1, "две": 2, "одну": 1, "и": ".",
    "нацело": "//", "степень": "**", "тысячу": 1000
}
dic_fl = {"одна": 1, "две": 2}  # дополнительный словарь для вывода вещественных чисел
float_endings = {'0': "ых", '1': "ая", '2': "ые", '3': "ые", '4': "ые", '5': "ых", '6': "ых", '7': "ых", '8': "ых",
                 '9': "ых"}  # окончания для дробей (пять сотЫХ, одна десятАЯ)
thousand_endings = {'1': "а", '2': "и", '3': "и", '4': "и", '5': "", '6': "", '7': "", '8': "", '9': "", '0': ""}  # окончания для тысяч
float_dic = {  # множители десятичных дробей до миллионных
    'десятая': 0.1, 'сотая': 0.01, 'тысячная': 0.001, 'десятитысячная': 0.0001, 'стотысячная': 0.00001,
    'миллионная': 0.000001,
    'десятую': 0.1, 'сотую': 0.01, 'тысячную': 0.001, 'десятитысячную': 0.0001, 'стотысячную': 0.00001,
    'миллионную': 0.000001,
    'десятые': 0.1, 'сотые': 0.01, 'тысячные': 0.001, 'десятитысячные': 0.0001, 'стотысячные': 0.00001,
    'миллионные': 0.000001,
    'десятых': 0.1, 'сотых': 0.01, 'тысячных': 0.001, 'десятитысячных': 0.0001, 'стотысячных': 0.00001,
    'миллионных': 0.000001
}
exception_list = ["на", "целых", "целая", "целые", "поделить", "с", "в", "возвести", "скобка", "скобочка"]  # список допустимых для использования в выражении слов. Иначе - "Некорректный ввод"

# аннотация
print('''\x1B[3m    Калькулятор целиком удовлетворяет условиям пунктов 1, 3, 4 - плюс 8 баллов сложности.
Дополнительно реализованная поддержка ввода/вывода дробей до миллионных включительно (часть пункта 2)
и диагностика ошибок (пункт 10).
Поддерживаются такие операции как:
сложение, вычитание, умножение, деление, целочисленное деление, деление с остатком, возведение в степень.
При этом возможна обработка сразу нескольких операций с правильной последовательностью действий.
В том числе поддерживается приоритизация операций при помощи скобок.
Поддерживается ввод чисел только меньше двух тысяч и вывод чисел до миллиона (не включая).
\x1B[23m
Операции: (в скобках обозначены не обязательные слова)
+ плюс
- минус
* умножить (на)
** (возвести в) степень
/ разделить (на)
// (поделить) нацело (на)
% (поделить с) остатком (на)
( (скобка/скобочка) открывается
) (скобка/скобочка) закрывается
. (целых/целая/целые) и
Внизу кода есть закомментированные выражения для тестов''')

calc(input("Введите строковое выражение: "))  # вводим наше выражение и вызываем calc

# выражения для тестов

"""calc("Восемь плюс десять")
calc("Пять минус четыре")
calc("Двести двадцать один умножить на пять")
calc("Восемьдесят один разделить на девять")
calc("Пятьсот семьдесят три поделить нацело на пятьдесят")
calc("Девятьсот девяносто девять поделить с остатком на сто")
calc("Три возвести в степень четыре")
calc("Пятьдесят восемь целых и сто пятьдесят три тысячных минус два умножить на пять")
calc("Одна целая и одна миллионная умножить на тысячу")
calc("Скобка открывается пять плюс четыре умножить на ноль целых и двадцать пять сотых скобка закрывается умножить на десять")
calc("Пять целых и одна миллионная плюс две целые и одна стотысячная")
calc("Скобка открывается один плюс два возвести в степень три умножить на двадцать пять скобка закрывается поделить нацело на два")
calc("Двести умножить на двести пятьдесят умножить на пятнадцать")
print('999999.999999 =', output('999999.999999'), '\n')
print(output('1000000000'))"""



восемьдесят две и восемьсот двадцать пять пятьсот пятьдесят пять миллионных
